In [26]:
# https://hobby-weighted.tistory.com/160

import pandas as pd
path = 'C:/Users/100do/python/승혜/archive/'
ytrain = pd.read_csv(path + 'y_train.csv')
xtrain = pd.read_csv(path + 'X_train_2.csv' )
xtest = pd.read_csv(path + 'X_test_2.csv')
xtest.head()

,cust_id,total_buy,max_buy,payback,main_product,where,days,buy_per_day,weekend_come,buy_period
0,3500,70900400,22000000,4050000.0,골프,부산본점,13,1.461538,0.789474,26
1,3501,310533100,38558000,48034700.0,농산물,잠실점,90,2.433333,0.369863,3
2,3502,305264140,14825000,30521000.0,가공식품,본 점,101,14.623762,0.083277,3
3,3503,7594080,5225000,NaN,주방용품,부산본점,5,2.000000,0.000000,47
4,3504,1795790,1411200,NaN,수산품,청량리점,3,2.666667,0.125000,8


In [27]:
# 데이터 전처리 없이 xgboost 사용할 경우를 비교해보자.
# 이상치 제거 안하고, ob라벨링, num 스케일링만 해서 학습시키면?
# 일단 null 값 제거는 동일하게 하자.
num = ['total_buy', 'max_buy','payback', 'days', 'buy_per_day','weekend_come','buy_period']
xtrain.info()

ob = ['main_product','where']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cust_id       3500 non-null   int64  
 1   total_buy     3500 non-null   int64  
 2   max_buy       3500 non-null   int64  
 3   payback       1205 non-null   float64
 4   main_product  3500 non-null   object 
 5   where         3500 non-null   object 
 6   days          3500 non-null   int64  
 7   buy_per_day   3500 non-null   float64
 8   weekend_come  3500 non-null   float64
 9   buy_period    3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB


In [28]:
xtrain = xtrain.fillna(xtrain.mean())
xtrain.isnull().sum()

cust_id         0
total_buy       0
max_buy         0
payback         0
main_product    0
where           0
days            0
buy_per_day     0
weekend_come    0
buy_period      0
dtype: int64

In [29]:
# nan값 처리
xtrain = xtrain.fillna(xtrain.mean())
xtest = xtest.fillna(xtrain.mean())

# ob 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for i in ob:
    encoder.fit(xtrain[i])
    xtrain[i] = encoder.transform(xtrain[i])
    xtest[i] = encoder.transform(xtest[i])

In [33]:
# 이상치 처리 코드 넣기  1. 일괄 이상치를 경계선 값으로 바꿈 2. 이상치 많이 나오는 데이터는 그냥 냅두기
#  
q3 = xtrain[num].quantile(q = 0.75)
q1 = xtrain[num].quantile(q = 0.25)
iqr = q3 - q1
up = q3 + iqr*1.5
xtrain[num] = xtrain[num][xtrain[num] < up ]
xtrain = xtrain.fillna(up)
#이상치 제거 하나 안하나 값 비슷하게 나옴

In [34]:
xtest[num] = xtest[num][xtest[num] < up ]
xtest = xtest.fillna(up)

In [38]:
ytrain['gender']

0       0
1       0
2       1
3       1
4       0
       ..
3495    1
3496    1
3497    0
3498    0
3499    0
Name: gender, Length: 3500, dtype: int64

In [48]:
# xtrain을 valid, train으로 split
import sklearn.model_selection
from sklearn.model_selection import train_test_split
#help(train_test_split)

xxtrain, xvalid, yytrain, yvalid = train_test_split(xtrain, ytrain, test_size=0.25, random_state=42, stratify = ytrain['gender'])
xxtrain.head()

# minmax scaling
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(xxtrain[num])
xxtrain[num] = scaler.transform(xxtrain[num])
xvalid[num] = scaler.transform(xvalid[num])
xtest[num] = scaler.transform(xtest[num])

<ipython-input-48-e49cbe8b25e8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xxtrain[num] = scaler.transform(xxtrain[num])
c:\users\100do\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<ipython-input-48-e49cbe8b25e8>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [49]:
import xgboost.sklearn
from xgboost.sklearn import XGBClassifier

In [50]:
xgb = XGBClassifier()
xcol = xxtrain.columns.difference(['cust_id'])
x = xxtrain[xcol]
y = yytrain['gender']
x2 = xvalid[xcol]
y2 = yvalid['gender']


xgb.fit(x, y)
ypred = xgb.predict(x2)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y2, ypred)

0.6068571428571429

<br>
<br>

# 기출문제 
고객이 주문한 물품의 정시 도착 여부 <br>


In [149]:
import pandas as pd
path = 'C:/Users/100do/python/승혜/archive/e_commerse_shipping_Data/'
df = pd.read_csv(path + 'Train.csv')
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [ ]:
# test set이 따로 없으니까 알아서 분리해야함.

1. 결측치 변환, 
2. 숫자형/ 글자형 컬럼 분리
3. 글자형은 sklearn.preprocessing에서 encodr 로 라벨링
4. 숫자형 컬럼 이상치 제거(한거 안한거 비교하기)
5. train/valid 분리
6. 숫자형 컬럼 이상치 제거 후 min max scailing
7. xgboost.sklearn 에서 classifier 찾아서 학습
5. metrics에서 accuracy_score 확인해보기

In [130]:
# 결측치 확인
df.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64

In [117]:
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,1,D,Flight,4,2,177,3,low,F,44,1233,1
1,2,F,Flight,4,5,216,2,low,M,59,3088,1
2,3,A,Flight,2,2,183,4,low,M,48,3374,1
3,4,B,Flight,3,3,176,4,medium,M,10,1177,1
4,5,C,Flight,2,2,184,3,medium,F,46,2484,1


In [152]:
# 숫자형 글자형 분리
df.info()

ob = ['Warehouse_block','Mode_of_Shipment', 'Product_importance', 'Gender']
num = df.columns.difference(ob)
num2 = num.difference(['ID','Reached.on.Time_Y.N'])
num_id = num.difference(['Reached.on.Time_Y.N'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10999 entries, 0 to 10998
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   10999 non-null  int64
 1   Warehouse_block      10999 non-null  int32
 2   Mode_of_Shipment     10999 non-null  int32
 3   Customer_care_calls  10999 non-null  int64
 4   Customer_rating      10999 non-null  int64
 5   Cost_of_the_Product  10999 non-null  int64
 6   Prior_purchases      10999 non-null  int64
 7   Product_importance   10999 non-null  int32
 8   Gender               10999 non-null  int32
 9   Discount_offered     10999 non-null  int64
 10  Weight_in_gms        10999 non-null  int64
 11  Reached.on.Time_Y.N  10999 non-null  int64
dtypes: int32(4), int64(8)
memory usage: 859.4 KB


In [153]:
# encoder로 글자형 라벨링
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for i in ob:
    encoder.fit(df[i])
    df[i] = encoder.transform(df[i])

In [154]:
df.describe()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,10999.00000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000,10999.000000
mean,5500.00000,2.333394,1.516865,4.054459,2.990545,210.196836,3.567597,1.346031,0.495863,13.373216,3634.016729,0.596691
std,3175.28214,1.490726,0.756894,1.141490,1.413603,48.063272,1.522860,0.631434,0.500006,16.205527,1635.377251,0.490584
min,1.00000,0.000000,0.000000,2.000000,1.000000,96.000000,2.000000,0.000000,0.000000,1.000000,1001.000000,0.000000
25%,2750.50000,1.000000,1.000000,3.000000,2.000000,169.000000,3.000000,1.000000,0.000000,4.000000,1839.500000,0.000000
50%,5500.00000,3.000000,2.000000,4.000000,3.000000,214.000000,3.000000,1.000000,0.000000,7.000000,4149.000000,1.000000
75%,8249.50000,4.000000,2.000000,5.000000,4.000000,251.000000,4.000000,2.000000,1.000000,10.000000,5050.000000,1.000000
max,10999.00000,4.000000,2.000000,7.000000,5.000000,310.000000,10.000000,2.000000,1.000000,65.000000,7846.000000,1.000000


In [155]:
#  숫자형 컬럼 이상치 제거는 하지말자
numdf = df[num2]
q3 = numdf[num2].quantile(q =0.75)
q1 = numdf[num2].quantile(q = 0.25)
iqr = q3-q1
up = q3 + iqr*1.5
down = q1 - iqr*1.5
numdf = numdf[(numdf<up)]
numdf2 = numdf.fillna(up)
numdf2 = numdf2[(numdf2>down)]
numdf3 = numdf2.fillna(down)

df[num2] = numdf3

In [156]:
# train test 분리
from sklearn import model_selection
from sklearn.model_selection import train_test_split

y = df['Reached.on.Time_Y.N']
x = df[num_id]
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.33, stratify = y)

In [159]:

xtest_id = xtest['ID']  # 테스트 결과 입력할때 필요함
xtest_id

xtrain = xtrain[num2]
xtest = xtest[num2]

In [160]:
# min max scailing
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(xtrain[num2])
xtrain[num2] = scaler.transform(xtrain[num2])
xtest[num2] = scaler.transform(xtest[num2])

In [161]:
from xgboost.sklearn import XGBClassifier

xgb = XGBClassifier()
xgb.fit(xtrain, ytrain)
ypred = xgb.predict(xtest)
ypred

array([1, 1, 1, ..., 0, 0, 1])

In [162]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred)

0.6581267217630854

In [167]:
awr_df = pd.DataFrame()
awr_df['ID'] = xtest_id
awr_df['predict'] = ypred
awr_df['real'] = ytest

In [168]:
awr_df

,ID,predict,real
699,700,1,1
6571,6572,1,1
8219,8220,1,1
2956,2957,1,1
7387,7388,0,0
...,...,...,...
1990,1991,1,1
3293,3294,0,0
3706,3707,0,0
7796,7797,0,0
